1973 IBM Selectric 721

https://typewriterdatabase.com/1973-ibm-selectric-721.19605.typewriter

In [1]:
%%writefile random_number.cpp
#include <sodium.h>
#include <stdio.h>
#include <vector>
#include <string>

int main(void)
{
    if (sodium_init() < 0) {
        printf("Error initializing libsodium\n");
        return 1;
    }

    uint32_t upper_bound = 128; // 2^7
    std::vector<std::string> existing_numbers = {"0000000", "0000001", "0000010", "0000011", 
                             "0000100", "0000101", "0000110", "0000111", 
                             "0001000", "0001010", "0001011", "0001100", "0001110", "0001111",
                             "0010000", "0010001", "0010010", 
                             "0010011", "0010100", "0010101", "0010110", 
                             "0010111", "0011000", "0011001", "0011010", 
                             "0011011", "0011100", "0011101", "0011110", 
                             "0011111", "0111100", "0111110",
                             "1011110", 
                             "1100000", "1111011", "1111100", "1111101", 
                             "1111110", "1111111"};


    std::string random_number_string;
    uint32_t random_number;
    bool number_exists = true;
    while (number_exists) {
        random_number = randombytes_uniform(upper_bound);
        random_number_string.clear();
        for (int i = 6; i >= 0; i--) {
            random_number_string += ((random_number >> i) & 1) + '0';
        }

        number_exists = false;
        for (const std::string& existing_number : existing_numbers) {
            if (existing_number == random_number_string) {
                number_exists = true;
                break;
            }
        }
    }

    printf("%s\n", random_number_string.c_str());

    return 0;
}



Writing random_number.cpp


In [2]:
!g++ random_number.cpp -o random_number -L /lib/x86_64-linux-gnu -lsodium

In [3]:
%%writefile binary_numbers.cpp
#include <iostream>
#include <fstream>
#include <cstring>
#include <unistd.h>

int main(int argc, char *argv[]) {
  if (argc < 2) {
    std::cerr << "Usage: " << argv[0] << " <filename>" << std::endl;
    return 1;
  }

  // Create a binary file with 7-bit ASCII encoding
  std::ofstream binaryFile(argv[1], std::ios::out | std::ios::binary);
  if (!binaryFile) {
    std::cerr << "Unable to create binary file." << std::endl;
    return 1;
  }

  bool continueRunning = true;
  while (continueRunning) {
    // Get binary data from the command system("./random_number")
    std::string binaryData;
    char buffer[256];
    memset(buffer, 0, sizeof(buffer));
    FILE* pipe = popen("./random_number", "r");
    if (!pipe) {
      std::cerr << "Unable to run command." << std::endl;
      return 1;
    }
    while (fgets(buffer, sizeof(buffer), pipe) != NULL) {
      binaryData += buffer;
    }
    pclose(pipe);

    // Append binary data to the binary file
    binaryFile.write(binaryData.c_str(), binaryData.length());

    std::cout << "."; // Replace with a short message to show that the program is running

    // Wait for a few seconds before repeating
    sleep(0); // Replace with the appropriate time interval
  }

  binaryFile.close();
  return 0;
}

Writing binary_numbers.cpp


In [4]:
!g++ binary_numbers.cpp -o binary_numbers

In [5]:
import time
import multiprocessing
import os

def task(process_num, binary_path, output_dir):
    filename = os.path.join(output_dir, f'file{process_num}.bin')
    if os.path.exists(filename):
        mode = 'ab'  # append if file exists
    else:
        mode = 'wb'  # write new file if not exists
    with open(filename, mode) as f:
        os.system(f'{binary_path} {filename}')

def monkey_simulation(proc, binary_path, output_dir):
    while True:
        processes = []
        for i in range(proc):
            p = multiprocessing.Process(target=task, args=(i+1, binary_path, output_dir))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()

        time.sleep(1)  # Wait for a second before starting a new set of processes



In [6]:
import os
from datetime import datetime

# Set the directory where you want to create the folder
directory = "/content/drive/MyDrive/Monkey/Simulation"

# Get the current date and time in the required format
now = datetime.now()
date_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create a new directory with the current date and time as its name
new_folder = os.path.join(directory, date_time)
if not os.path.exists(new_folder):
    os.makedirs(new_folder)
    print(f"Directory created: {new_folder}")
else:
    print(f"Directory {new_folder} already exists")


Directory created: /content/drive/MyDrive/Monkey/Simulation/2023-03-03_15-35-58


In [ ]:
monkey_simulation(120, '/content/binary_numbers', new_folder)